In [16]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

import requests
from io import StringIO

url = 'https://raw.githubusercontent.com/kierajclarke/Final-Data-Project/main/student_math_clean.csv'
response = requests.get(url)
data = pd.read_csv(StringIO(response.text))


In [32]:
unique_sex_values = data['sex'].unique()
print(unique_sex_values)

['F' 'M']


In [33]:
# Mapping 'M' to 'Male' and 'F' to 'Female'
data['sex_numeric'] = data['sex'].map({'F': 0, 'M': 1})  

First we select columns related to academic metrics and sex and preprocess the data. Since our aim is to analyze the data of highschool students, we want to limit the age range of the students so that outlying ages do not skew accuracy.

In [42]:
# Filter the dataset for ages between 15 and 18, including selected columns
data = data.loc[(data['age'] >= 15) & (data['age'] <= 18), ['age', 'sex', 'grade_1', 'grade_2', 'final_grade']]


# Convert grades to percentages
# Multiply by 5 to convert from 0-20 to 0-100
data[['grade_1', 'grade_2', 'final_grade']] = data[['grade_1', 'grade_2', 'final_grade']] * 5  

# Display the first few rows with grades converted to percentages
print(data.head())

   age sex  grade_1  grade_2  final_grade
0   18   F       25       30           30
1   17   F       25       25           30
2   15   F       35       40           50
3   15   F       75       70           75
4   16   F       30       50           50


Now we want to examine the correlation between male and female

In [43]:
# Separate the data by gender
female_data = data[data['sex'] == 'F']
male_data = data[data['sex'] == 'M']

# Calculate the correlations for females
female_correlation = female_data[['grade_1', 'grade_2', 'final_grade']].corr()
print("Correlation for Female students:")
print(female_correlation)

# Calculate the correlations for males
male_correlation = male_data[['grade_1', 'grade_2', 'final_grade']].corr()
print("\nCorrelation for Males students:")
print(male_correlation)


Correlation for Female students:
              grade_1   grade_2  final_grade
grade_1      1.000000  0.839361      0.78435
grade_2      0.839361  1.000000      0.89311
final_grade  0.784350  0.893110      1.00000

Correlation for Males students:
              grade_1   grade_2  final_grade
grade_1      1.000000  0.857281     0.825429
grade_2      0.857281  1.000000     0.928189
final_grade  0.825429  0.928189     1.000000


In terms of differential privacy, can we get the same results

In [44]:
# Example of adding Laplace noise to sex-academic metrics correlations
epsilon = 1.0 
sensitivity = 1.0  

noisy_sex_grades_correlation = sex_grades_correlation['sex_numeric'].apply(
    lambda x: laplace_mech(x, sensitivity, epsilon)
)

print(noisy_sex_grades_correlation)


sex_numeric    1.131098
grade_1        0.670350
grade_2       -0.842761
final_grade    0.769353
Name: sex_numeric, dtype: float64


Computing differentially private means for Male and Female students

In [45]:
# Calculate means for each gender
mean_grade_male = data[data['sex'] == 'M']['final_grade'].mean()
mean_grade_female = data[data['sex'] == 'F']['final_grade'].mean()

print("Mean final grade for Male students:", mean_grade_male)
print("Mean final grade for Female students:", mean_grade_female)


Mean final grade for Male students: 55.46242774566474
Mean final grade for Female students: 50.284974093264246


Now using Laplace Mechanism to create noise 

In [54]:

# Set privacy parameters
epsilon = 1.0  
sensitivity = data['final_grade'].max() - data['final_grade'].min()

# Apply Laplace noise to means
noisy_mean_grade_male = laplace_mech(mean_grade_male, sensitivity, epsilon)
noisy_mean_grade_female = laplace_mech(mean_grade_female, sensitivity, epsilon)

print("Noisy mean final grade for Male students:", noisy_mean_grade_male)
print("Noisy mean final grade for Female students:", noisy_mean_grade_female)


Noisy mean final grade for Male students: 102.30098390568511
Noisy mean final grade for Female students: 128.15125678407608


Now using Gaussian Mechanism to create noise

In [55]:
# Apply Gaussian noise to means for male and female grades
delta = 1e-5
    
noisy_mean_grade_male = gaussian_mech(mean_grade_male, sensitivity, epsilon, delta)
noisy_mean_grade_female = gaussian_mech(mean_grade_female, sensitivity, epsilon, delta)

print("Noisy mean grades for Male students:")
print(noisy_mean_grade_male)

print("\nNoisy mean grades for Female students:")
print(noisy_mean_grade_female)

Noisy mean grades for Male students:
-333.4129389475042

Noisy mean grades for Female students:
657.1379654883984
